In [15]:
%run lm_ode.ipynb

In [2]:
import matplotlib.pyplot as plt
from scipy.linalg import hilbert
from mpl_toolkits.mplot3d import Axes3D 

# Model 1: Homogeneous gas phase reaction model
$$2NO+O_2\leftrightarrow2NO_2$$
is described by the following equation
$$\frac{dy}{dt}=k_1(126.2-y)(91.9-y)^2-k_2y^2;~~y(0)=0$$

In [3]:
def model1(y,k):
    return k[0]*(126.2-y)*(91.9-y)**2-k[1]*y**2

## Scaling for parameter estimation
$$\frac{du}{d\bar{t}}=k_1t_cy_c^2(126.2/y_c-u)(91.9/y_c-u)^2-t_ck_2y_cu^2$$
Let $y_c=91.9$ and $t_c=t_\text{end}$,
$$\frac{du}{d\bar{t}}=91.9^2t_\text{end}k_1(126.2/91.9-u)(1-u)^2-91.9t_\text{end}k_2u^2$$
$$\frac{du}{d\bar{t}}=\bar{k}_1(126.2/91.9-u)(1-u)^2-\bar{k}_2u^2$$
where $k_1=\frac{\bar{k}_1}{91.9^2t_\text{end}}$ and $k_2=\frac{\bar{k}_2}{91.9t_\text{end}}$.

In [4]:
def model1_scaled(u,kbar):
    dudt = kbar[0]*(126.2/91.9-u)*(1-u)**2-kbar[1]*u**2
    return dudt

In [5]:
yhat=np.array([0,1.4,6.3,10.5,14.2,17.6,21.4,23.0,27.0,30.5,34.4,38.8,41.6,43.5,45.3])
y0 = yhat[0]
t = np.array([0,1,2,3,4,5,6,7,9,11,14,19,24,29,39])
N = np.size(t)
k = np.array([0.4577e-5,0.2796e-3])
k0 = np.array([1e-6,1e-6])
yc = 91.9
uhat = yhat/yc
u0 = uhat[0]
tc = t[-1]
tbar = t/tc
Q = np.eye(1)
Kc = np.array([[91.9**2*tc,0],[0,91.9*tc]])
kbar = Kc@k
k0bar = Kc@k0

## Paramter estimation with given paramters

In [10]:
k_est = gnlm(model1,yhat,Q,k,t,[1e-3,1e-8,1e-8,100])

Iter | Obj func | step size | gradient |   mu   |   rho
    0|1.0933e+01|   Not cal |  Not cal | 1.3e+11| Not cal
    1|1.0933e+01|  1.262e-10|  5.70e+03| 2.6e+11| Not cal
    2|1.0933e+01|  7.376e-11|  5.70e+03| 1.0e+12| Not cal
    3|1.0933e+01|  4.603e-11|  1.52e+01| 1.5e+12| 1.3e-01
    4|1.0933e+01|  1.034e-11|  1.53e+01| 4.9e+11| 3.9e+03
    5|1.0933e+01|  3.103e-11|  1.53e+01| 9.8e+11| 3.9e+03
    6|1.0933e+01|  1.552e-11|  1.53e+01| 3.9e+12| 3.9e+03
    7|1.0933e+01|  3.883e-12|  1.53e+01| 3.1e+13| 3.9e+03
    8|1.0933e+01|  4.875e-13|  1.53e+01| 3.1e+13| 3.9e+03


In [11]:
k_est

[array([4.57704357e-06, 2.79600025e-04]),
 array([[ 0.        ,  4.54931465,  8.52416818, 12.02638888, 15.13394993,
         17.90778445, 20.39635284, 22.63879046, 26.50790522, 29.7121704 ,
         33.568332  , 38.19737929, 41.32898214, 43.48877739, 46.05875815]]),
 [8, 'h']]

In [16]:
k_est = gnlm(model1,yhat,Q,k0,t,[1e-3,1e-8,1e-8,100])

Iter | Obj func | step size | gradient |   mu   |   rho
    0|2.0059e+03|   Not cal |  Not cal | 1.2e+12| Not cal
    1|2.3718e+02|  1.757e-06|  3.62e+08| 4.0e+11| 9.6e-01
    2|4.8949e+01|  1.163e-06|  4.11e+07| 1.3e+11| 1.0e+00
    3|4.4385e+01|  2.042e-06|  2.04e+06| 4.4e+10| 1.0e+00
    4|4.2666e+01|  6.180e-06|  4.34e+05| 1.5e+10| 1.0e+00
    5|3.8218e+01|  1.713e-05|  9.17e+05| 4.9e+09| 9.9e-01
    6|2.9152e+01|  4.123e-05|  1.60e+06| 1.6e+09| 9.8e-01
    7|1.8051e+01|  7.358e-05|  1.36e+06| 5.5e+08| 9.6e-01
    8|1.2050e+01|  8.069e-05|  2.15e+05| 1.8e+08| 9.6e-01
    9|1.0974e+01|  4.603e-05|  1.70e+05| 6.1e+07| 9.6e-01
   10|1.0933e+01|  1.058e-05|  4.62e+04| 2.0e+07| 9.6e-01
   11|1.0933e+01|  5.501e-07|  1.99e+03| 1.2e+07| 8.7e-01
   12|1.0933e+01|  6.892e-09|  4.57e+01| 4.1e+06| 1.3e+00
   13|1.0933e+01|  3.224e-10|  4.29e+01| 1.4e+06| 2.5e+05
   14|1.0933e+01|  2.961e-10|  1.35e+00| 4.6e+05| 1.3e+04
   15|1.0933e+01|  4.234e-12|  1.35e+00| 9.1e+05| 1.3e+04
   16|1.0933e+01

In [13]:
k_est

[array([4.57708752e-06, 2.79623754e-04]),
 array([[ 0.        ,  4.54935527,  8.52423873, 12.02648078, 15.13405619,
         17.90789928, 20.39647141, 22.63890869, 26.50801291, 29.71225751,
         33.56837535, 38.19733047, 41.32883346, 43.4885316 , 46.05834563]]),
 [19, 'h']]